In [1]:
import pandas as pd

In [5]:
def combine_year(df_list, year, path):
    num = 1
    while num:
        try:
            temp_df = pd.read_csv(path + f'mlb_war_ratings_{year}{num}.csv')
            df_list.append(temp_df)
            num += 1
        except:
            return df_list
            break

In [7]:
df_list = []
path = 'unclean_data/mlb_war_by_year/'
years = [i for i in range(2011, 2019)]
for year in years:
    combine_year(df_list, year, path)

In [9]:
df = pd.concat(df_list)

In [13]:
temp = df

In [17]:
df.dropna(how = 'any', subset = ['WAR/pos'], inplace = True)

In [20]:
df = df[['Player', 'WAR/pos', 'Year']]
df.head()

,Player,WAR/pos,Year
0,Jose Bautista\bautijo02,8.3,2011
1,Jacoby Ellsbury\ellsbja01,8.3,2011
2,Matt Kemp\kempma01,8.0,2011
3,Dustin Pedroia\pedrodu01,8.0,2011
4,Ryan Braun\braunry02,7.7,2011


In [21]:
for i in range(len(df)):
    df.iat[i, 0] = df.iat[i, 0].split('\\')[0]

In [24]:
df = df.rename(columns = {'WAR/pos': 'war', 'Year':'year', 'Player':'player'})

In [26]:
df.to_csv('mlb_player_war_year.csv', index = False)